In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.6 MB/s 
     |████████████████████████████████| 448 kB 42.0 MB/s 
     |████████████████████████████████| 86 kB 3.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Komoran

In [ ]:
tomato = pd.read_csv("/content/drive/MyDrive/선대팀_주제분석/tomato_news.csv")
daepa = pd.read_csv("/content/drive/MyDrive/선대팀_주제분석/daepa_news.csv")
carrot = pd.read_csv("/content/drive/MyDrive/선대팀_주제분석/carrot_news.csv")

In [ ]:
# 각 농산물에 특화된 기사 
tomato = tomato.drop("Unnamed: 0", axis = 1)
daepa = daepa.drop("Unnamed: 0", axis = 1)
carrot = carrot.drop("Unnamed: 0", axis = 1)

In [ ]:
tomato.head()

,date,press,title,content,link
0,2016.04.29.,한겨레,"모종은 옮겨 심고, 잡풀은 없애고","마침 곡우에 단비가 내렸으니, 초보 농부도 ‘개불알에 쌀밥 붙으리라’며 풍작을 ...",http://www.seouland.com/arti/society/society_g...
1,2016.05.26.,경향신문,“먹을 토마토도 없는데… 던져서 버리다니”,나이지리아가 스페인 토마토 축제를 걸고넘어지게 된 계기는 유례없는 토마토 흉작이...,http://news.khan.co.kr/kh_news/khan_art_view.h...
2,2016.08.08.,인민망,"닝샤 토마토 농가 비상, 토마토 500g에 0.1위안",토마토 농사는 풍작을 이뤘지만 현지 농민들은 어찌할 바를 몰라했다. 작년 500...,http://kr.people.com.cn/n3/2016/0808/c203281-9...
3,2016.10.30.,리얼푸드,"[리얼푸드]“韓, 日 토마토 수출 늘리려면 상품성 높여야”","주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일...",http://www.realfoods.co.kr/realfoods/view.php?...
4,2016.10.31.,헤럴드경제,“토마토 日수출 늘리려면 고부가상품 개발을”,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일...",http://news.heraldcorp.com/view.php?ud=2016103...


In [ ]:
# 모든 농산물에 공통적으로 존재하는 기사 
# 분석 시에 각 농산물 + 공통기사로 각 농산물의 감성지수 산출

common = pd.read_csv("/content/drive/MyDrive/선대팀_주제분석/common_news.csv")
common = common.drop('Unnamed: 0', axis = 1)

In [ ]:
# 필요한 열만 추출
common = common.loc[:, ['date','content']]
tomato = tomato.loc[:, ['date','content']]
daepa = daepa.loc[:, ['date','content']]
carrot = carrot.loc[:, ['date','content']]

In [ ]:
tomato

,date,content
0,2016.04.29.,"마침 곡우에 단비가 내렸으니, 초보 농부도 ‘개불알에 쌀밥 붙으리라’며 풍작을 ..."
1,2016.05.26.,나이지리아가 스페인 토마토 축제를 걸고넘어지게 된 계기는 유례없는 토마토 흉작이...
2,2016.08.08.,토마토 농사는 풍작을 이뤘지만 현지 농민들은 어찌할 바를 몰라했다. 작년 500...
3,2016.10.30.,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일..."
4,2016.10.31.,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일..."
...,...,...
304,2020.09.28.,"내외뉴스통신, NBNNEWS 이성원 기자 올 여름 한반도를 덮친 태풍과 긴 장마..."
305,2020.09.28.,"올여름 한반도를 강타한 태풍과 장마 때문에 토마토 수급에 차질을 빚자, 햄버거 ..."
306,2020.09.28.,장마와 태풍으로 토마토 수급에 차질이 빚어지고 있기 때문이다. 소비자들은 “밀가...
307,2020.09.28.,"한편, 햄버거 업계는 위생 문제 뿐만 아니라 장마와 태풍의 영향으로 토마토 수급..."


In [ ]:
# snu 감성사전은 kkma를 이용해 작업할 것
kkma = Kkma()

In [ ]:
def getWords(data) : 
  omit_list = list()
  word_list = list()
  for i in range(len(data)) : 
    try : 
      words = kkma.pos(data.iloc[i,1])
      word_list.append(words)
    except : # 가끔 내용이 아예 존재하지 않는 기사가 있으므로 이를 예외처리 하고 해당 행을 제외하기 위해 i값을 기록
      print(i) 
      omit_list.append(i)
      continue
  return word_list, omit_list

In [ ]:
tomato_run = getWords(tomato)
tomato_words = tomato_run[0]
tomato_omits = tomato_run[1]

In [ ]:
daepa_run = getWords(daepa)
daepa_words = daepa_run[0]
daepa_omits = daepa_run[1]

In [ ]:
carrot_run = getWords(carrot)
carrot_words = carrot_run[0]
carrot_omits = carrot_run[1]

In [ ]:
common_run = getWords(common) # 출력되는 숫자들은 아무 내용이 없는 행 
common_words = common_run[0]
common_omits = common_run[1]

1392
1439
1541
1557
1570
1571
15262


In [ ]:
common = common.drop(common_omits, axis = 0) # 아무 내용이 없는 기사 제거 

In [ ]:
common.to_csv("/content/drive/MyDrive/선대팀_주제분석/common_news_ver2.csv", index = False) # 혹시 모를 오류를 대비해서 파일을 미리 저장해두기

In [ ]:
def make_text(text_list) : 
  my_text = ''
  for sentence in text_list: 
    for item in sentence : 
      word = item[0]
      shape = item[1]
      if item == sentence[-1] :
        my_text += word+shape
      else : 
        my_text += word+shape+'/'
    my_text += '\n'
  return my_text

In [ ]:
# konlpy 특성상 매우 오래 걸리므로 혹시 모를 상황을 대비해
# 지금까지 해놓은 작업을 중단하더라도 나중에 처음부터 다시 하지 않고 중간부터 이어가기위해 현재 작성해놓은 상태를
# 파일로 저장을 해두기

tomato_text = make_text(tomato_words)
daepa_text = make_text(daepa_words)
carrot_text = make_text(carrot_words)
common_text = make_text(common_words)

tomato_txt = open("tomato.txt", 'w')
tomato_txt.write(tomato_text)
daepa_txt = open("daepa.txt",'w')
daepa_txt.write(daepa_text)
carrot_txt = open("carrot.txt",'w')
carrot_txt.write(carrot_text)
common_txt = open("common.txt",'w')
common_txt.write(common_text)

tomato_txt.close()
daepa_txt.close()
carrot_txt.close()
common_txt.close()

In [ ]:
tomato_text # 파일로 저장한 글 예시 

'마침MAG/곡우NNG/에JKM/단비NNG/가JKS/내리VV/었EPT/으니ECD/,SP/초보NNG/농부NNG/도JX/‘SS/개NNG/불알NNG/에JKM/쌀밥NNG/붙VV/으리라EFN/’SS/며JC/풍작NNG/을JKO/기대NNG/하XSV/ㄹETD/법하NNG/이VCP/다EFN/.SF/잎채소NNG/는JX/이미MAG/곡우NNG/를JKO/전후NNG/하XSV/어ECS/넣VV/었EPT/으니ECD/고추NNG/,SP/토마토NNG/,SP/가지NNG/,SP/호박NNG/,SP/오이NNG/,SP/생강NNG/등NNB/뿌리NNG/와JC/열매NNG/채소NNG/의JKG/모종NNG/을JKO/옮기VV/어ECS/심VV/어야ECD/...SE\n나이NNG/지리NNG/아가NNG/스페인NNG/토마토NNG/축제NNG/를JKO/걸VV/고ECE/넘어지VV/게ECD/되VV/ㄴETD/계기NNG/는JX/유례없VA/는ETD/토마토NNG/흉작NNG/이VCP/다EFN/.SF/나NP/이지리NNG/아XSN/북쪽NNG/카NNG/두JX/나NP/지역NNG/에서JKM/는JX/‘SS/투타NNG/앱UN/솔NNG/로JKM/타VV/아ECS/’SS/라NNG/불리VV/는ETD/나방NNG/때문NNB/에JKM/토마토NNG/생산량NNG/이JKS/80NR/%SW/줄VV/었EPT/다EFN/.SF/투타NNG/앱UN/솔NNG/로JKM/타VV/는ETD/...SE\n토마토NNG/농사NNG/는JX/풍작NNG/을JKO/이루VV/었EPT/지만ECE/현지NNG/농민NNG/들XSN/은JX/어찌하VV/ㄹETD/바NNB/를JKO/몰VV/ㄹ라ECD/하VV/었EPT/다EFN/.SF/작년NNG/500NR/gOL/에JKM/1NR/~SO/1.5NR/위안NNM/정도NNG/하VV/더EPT/ㄴETD/토마토NNG/가격NNG/이JKS/올해NNG/는JX/0.1NR/~SO/0.2NR/위안NNM/까지JX/떨어지VV/었EPT/기ETN/때문NNB/이VCP/다EFN/.SF/크VA/ㄴETD/폭NNG/으로JKM/떨어지VV/ㄴETD/토마토NNG/가격NNG/때문NNB/에JKM/많VA/은ETD/농민NNG

In [ ]:
import re # 정규표현식

In [ ]:
korean = re.compile('[가-핳]+')

In [ ]:
# 우리가 필요한 한글만 추출하기

def getOnlyWords(data) : 
  sentence_list = list()
  for sentence in data : 
    word_list = list()
    for tuples in sentence : 
      word = tuples[0]
      if len(korean.findall(word)) == 0 : 
        continue
      else : 
        word_list.append(tuples)
    sentence_list.append(word_list)
  return sentence_list

In [ ]:
tomato_origin = tomato_words.copy()
daepa_origin = daepa_words.copy()
carrot_origin = carrot_words.copy()
common_origin = common_words.copy()

In [ ]:
tomato_words = getOnlyWords(tomato_words)
daepa_words = getOnlyWords(daepa_words)
carrot_words = getOnlyWords(carrot_words)
common_words = getOnlyWords(common_words)

In [ ]:
# 감성사전 불러오기
snu_dict = pd.read_csv("/content/drive/MyDrive/선대팀_주제분석/snu_감성사전_new.csv")
snu_dict

,words,original_words,value
0,가JKS,가*/JKS,POS
1,가JKS/있VV,가*/JKS;있/VV,POS
2,가JKS/있VV/었EP,가*/JKS;있/VV;었/EP,POS
3,가VV,가*/VV,POS
4,가VV/다EF,가*/VV;ㄴ다*/EF,POS
...,...,...,...
16357,들VA/ETM것/NNB,힘들/VA;ㄹ/ETM;것/NNB,NEG
16358,들VA/ETM때/NNG,힘들/VA;ㄹ/ETM;때/NNG,NEG
16359,차VA,힘차/VA,NEG
16360,차VA/ETM,힘차/VA;ㄴ/ETM,NEG


In [ ]:
snu_dict_byindex = snu_dict.set_index('words')
snu_dict_byindex = snu_dict_byindex.drop('original_words', axis = 1)
snu_dict = snu_dict_byindex.to_dict()
snu_dict = snu_dict['value']

In [ ]:
def getUnitSentiment(word_list) : # 한 개의 조합에 대한 단어 일치 여부 검사 
  pos_list = list()
  neg_list = list()
  for sentence in word_list : 
    length = len(sentence)
    pos = 0
    neg = 0
    for i in range(length) :
      text = sentence[i][0]+sentence[i][1]
      if text in list(snu_dict.keys()) : 
        if snu_dict[text] == 'POS' : 
          pos +=1
        elif snu_dict[text] == 'NEG' :
          neg +=1
    pos_list.append(pos)
    neg_list.append(neg)
  return pos_list, neg_list

In [ ]:
def getDoubleSentiment(word_list) : # 두 개의 조합에 대한 단어 일치 여부 검사 
  pos_list = list()
  neg_list = list()
  for sentence in word_list : 
    length = len(sentence)
    pos = 0
    neg = 0
    for i in range(length-1) :
      text = sentence[i][0]+sentence[i][1]+ '/' + sentence[i+1][0]+sentence[i+1][1]
      if text in list(snu_dict.keys()) : 
        if snu_dict[text] == 'POS' : 
          pos +=1
        elif snu_dict[text] == 'NEG' :
          neg +=1
    pos_list.append(pos)
    neg_list.append(neg)
  return pos_list, neg_list

In [ ]:
def getTripleSentiment(word_list) : # 세 개의 조합에 대한 단어 일치 여부 검사 
  pos_list = list()
  neg_list = list()
  for sentence in word_list : 
    length = len(sentence)
    pos = 0
    neg = 0
    for i in range(length-2) :
      text = sentence[i][0]+sentence[i][1]+ '/' + sentence[i+1][0]+sentence[i+1][1] + '/' + sentence[i+2][0] + sentence[i+2][1]
      if text in list(snu_dict.keys()) : 
        if snu_dict[text] == 'POS' : 
          pos +=1
        elif snu_dict[text] == 'NEG' :
          neg +=1
    pos_list.append(pos)
    neg_list.append(neg)
  return pos_list, neg_list

In [ ]:
tomato_unit_pos, tomato_unit_neg = getUnitSentiment(tomato_words)
daepa_unit_pos, daepa_unit_neg = getUnitSentiment(daepa_words)
carrot_unit_pos, carrot_unit_neg = getUnitSentiment(carrot_words)
common_unit_pos, common_unit_neg = getUnitSentiment(common_words)

In [ ]:
tomato_double_pos, tomato_double_neg = getDoubleSentiment(tomato_words)
daepa_double_pos, daepa_double_neg = getDoubleSentiment(daepa_words)
carrot_double_pos, carrot_double_neg = getDoubleSentiment(carrot_words)
common_double_pos, common_double_neg = getDoubleSentiment(common_words)

In [ ]:
tomato_triple_pos, tomato_triple_neg = getTripleSentiment(tomato_words)
daepa_triple_pos, daepa_triple_neg = getTripleSentiment(daepa_words)
carrot_triple_pos, carrot_triple_neg = getTripleSentiment(carrot_words)
common_triple_pos, common_triple_neg = getTripleSentiment(common_words)

In [ ]:
tomato['pos1'] = tomato_unit_pos
tomato['neg1'] = tomato_unit_neg
daepa['pos1'] = daepa_unit_pos
daepa['neg1'] = daepa_unit_neg
carrot['pos1'] = carrot_unit_pos
carrot['neg1'] = carrot_unit_neg
common['pos1'] = common_unit_pos
common['neg1'] = common_unit_neg

In [ ]:
tomato['pos2'] = tomato_double_pos
tomato['neg2'] = tomato_double_neg
daepa['pos2'] = daepa_double_pos
daepa['neg2'] = daepa_double_neg
carrot['pos2'] = carrot_double_pos
carrot['neg2'] = carrot_double_neg
common['pos2'] = common_double_pos
common['neg2'] = common_double_neg

In [ ]:
tomato['pos3'] = tomato_triple_pos
tomato['neg3'] = tomato_triple_neg
daepa['pos3'] = daepa_triple_pos
daepa['neg3'] = daepa_triple_neg
carrot['pos3'] = carrot_triple_pos
carrot['neg3'] = carrot_triple_neg
common['pos3'] = common_triple_pos
common['neg3'] = common_triple_neg

In [ ]:
# 모든 조합을 다 더하면 우리가 원하는 최종 결과
tomato['pos'] = tomato['pos1'] + tomato['pos2'] + tomato['pos3']
tomato['neg'] = tomato['neg1'] + tomato['neg2'] + tomato['neg3']
daepa['pos'] = daepa['pos1'] + daepa['pos2'] + daepa['pos3']
daepa['neg'] = daepa['neg1'] + daepa['neg2'] + daepa['neg3']
carrot['pos'] = carrot['pos1'] + carrot['pos2'] + carrot['pos3']
carrot['neg'] = carrot['neg1'] + carrot['neg2'] + carrot['neg3']
common['pos'] = common['pos1'] + common['pos2'] + common['pos3']
common['neg'] = common['neg1'] + common['neg2'] + common['neg3'] 

In [ ]:
tomato # 결과 예시 

,date,content,pos2,neg2,pos3,neg3,pos1,neg1,pos,neg
0,2016.04.29.,"마침 곡우에 단비가 내렸으니, 초보 농부도 ‘개불알에 쌀밥 붙으리라’며 풍작을 ...",3,0,1,0,15,6,19,6
1,2016.05.26.,나이지리아가 스페인 토마토 축제를 걸고넘어지게 된 계기는 유례없는 토마토 흉작이...,1,0,0,0,8,7,9,7
2,2016.08.08.,토마토 농사는 풍작을 이뤘지만 현지 농민들은 어찌할 바를 몰라했다. 작년 500...,0,1,0,0,12,9,12,10
3,2016.10.30.,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일...",0,1,0,0,9,12,9,13
4,2016.10.31.,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일...",0,1,0,0,9,12,9,13
...,...,...,...,...,...,...,...,...,...,...
304,2020.09.28.,"내외뉴스통신, NBNNEWS 이성원 기자 올 여름 한반도를 덮친 태풍과 긴 장마...",0,1,0,0,7,7,7,8
305,2020.09.28.,"올여름 한반도를 강타한 태풍과 장마 때문에 토마토 수급에 차질을 빚자, 햄버거 ...",2,1,0,0,9,8,11,9
306,2020.09.28.,장마와 태풍으로 토마토 수급에 차질이 빚어지고 있기 때문이다. 소비자들은 “밀가...,0,0,0,0,8,9,8,9
307,2020.09.28.,"한편, 햄버거 업계는 위생 문제 뿐만 아니라 장마와 태풍의 영향으로 토마토 수급...",1,1,0,0,8,9,9,10


In [ ]:
# 중간 저장
tomato.to_csv("/content/drive/MyDrive/선대팀_주제분석/tomato_sentiment.csv", index = False)
daepa.to_csv("/content/drive/MyDrive/선대팀_주제분석/daepa_sentiment.csv", index = False)
carrot.to_csv("/content/drive/MyDrive/선대팀_주제분석/carrot_sentiment.csv", index = False)
common.to_csv("/content/drive/MyDrive/선대팀_주제분석/common_sentiment.csv", index = False)

In [ ]:
tomato = tomato.drop(['pos1','pos2','pos3','neg1','neg2','neg3'], axis = 1)
daepa = daepa.drop(['pos1','pos2','pos3','neg1','neg2','neg3'], axis = 1)
carrot = carrot.drop(['pos1','pos2','pos3','neg1','neg2','neg3'], axis = 1)
common = common.drop(['pos1','pos2','pos3','neg1','neg2','neg3'], axis = 1)

In [ ]:
tomato.head()

,date,content,pos,neg
0,2016.04.29.,"마침 곡우에 단비가 내렸으니, 초보 농부도 ‘개불알에 쌀밥 붙으리라’며 풍작을 ...",19,6
1,2016.05.26.,나이지리아가 스페인 토마토 축제를 걸고넘어지게 된 계기는 유례없는 토마토 흉작이...,9,7
2,2016.08.08.,토마토 농사는 풍작을 이뤘지만 현지 농민들은 어찌할 바를 몰라했다. 작년 500...,12,10
3,2016.10.30.,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일...",9,13
4,2016.10.31.,"주로 샐러드나 샌드위치, 패스트푸드점 등지의 업무용 수요로 활용되며, 일부는 일...",9,13


In [ ]:
def countWord(words) : 
  word_list = list()
  for sentence in words : 
    word_list.append(len(sentence))
  return word_list

In [ ]:
tomato['word_counts'] = countWord(tomato_words)
daepa['word_counts'] = countWord(daepa_words)
carrot['word_counts'] = countWord(carrot_words)
common['word_counts'] = countWord(common_words)

In [ ]:
# 감성지수 산출을 위해 긍정, 부정, 합을 구해서 데이터 프레임으로 저장
tomato_sum = tomato.groupby('date').sum()
daepa_sum = daepa.groupby('date').sum()
carrot_sum = carrot.groupby('date').sum()
common_sum = common.groupby('date').sum()

In [ ]:
tomato_sum

,pos,neg,word_counts
date,,,
2016.04.29.,19,6,54
2016.05.26.,9,7,54
2016.08.08.,12,10,52
2016.10.30.,9,13,47
2016.10.31.,9,13,48
...,...,...,...
2020.09.24.,252,229,1230
2020.09.25.,175,111,717
2020.09.26.,64,50,288


In [ ]:
full_data = pd.read_csv("/content/drive/MyDrive/선대팀_주제분석/full_data.csv", encoding = 'cp949') # 감성지수 제외하고 수집한 최종 데이터

In [ ]:
full_data.head()

,외국인노동자수,date,요일,강원영동강수량.mm.,강원영서강수량.mm.,경남강수량.mm.,경북강수량.mm.,서울경기강수량.mm.,전국강수량.mm.,전남강수량.mm.,전북강수량.mm.,제주강수량.mm.,충남강수량.mm.,충북강수량.mm.,강원영동평균기온...,강원영서평균기온...,경남평균기온...,경북평균기온...,서울경기평균기온...,전국평균기온...,전남평균기온...,전북평균기온...,제주평균기온...,충남평균기온...,충북평균기온...,유가,중국화폐종가,중국화폐시가,명절,태풍,대파거래량,대파가격,당근거래량,당근가격,토마토거래량,토마토가격
0,4363,2016.1.4,월요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.4,3.0,6.7,6.0,2.3,5.3,7.2,5.5,12.0,4.0,4.0,37.22,182.14,180.85,0,0,994328.1,1716,558950.2,794,291057.0,1834
1,4363,2016.1.5,화요일,0.0,0.0,0.5,0.0,0.0,0.5,2.4,0.0,5.9,0.0,0.0,1.8,-2.3,2.4,1.5,-2.2,0.6,2.8,0.6,8.6,-0.9,-1.4,36.42,182.57,181.95,0,0,787716.0,1715,444353.7,763,194626.5,1833
2,4363,2016.1.6,수요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,-3.7,2.6,1.0,-1.1,0.7,3.6,1.2,8.3,-0.1,-1.7,34.23,183.10,182.40,0,0,793584.9,1620,396170.4,787,178811.0,1999
3,4363,2016.1.7,목요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,-3.7,2.4,0.7,-2.7,0.4,3.2,1.4,6.7,0.2,-1.9,33.75,181.60,182.91,0,0,733312.0,1545,241115.0,770,107346.3,1921
4,4363,2016.1.8,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.2,0.1,0.1,0.0,-0.3,-5.2,1.4,-0.3,-2.6,-0.5,2.2,0.5,5.7,-0.5,-2.9,33.55,183.12,181.44,0,0,797767.3,1488,359789.0,705,190624.0,2107


In [ ]:
# 필요한 열을 만들어주기
full_data['pos'] = 0
full_data['neg'] = 0
full_data['counts'] = 0

In [ ]:
# 날짜 포맷을 바꿔주기

def changeDate(date) : 
  date_list = date.split('.')
  year = date_list[0]
  month = date_list[1]
  day = date_list[2]

  if int(month) < 10 : 
    month = '0'+month
  if int(day) < 10:
    day = '0' + day
  
  return year+'.'+month+'.'+day

In [ ]:
full_data['date'] = full_data['date'].apply(lambda x: changeDate(x))

In [ ]:
full_data.head()

,외국인노동자수,date,요일,강원영동강수량.mm.,강원영서강수량.mm.,경남강수량.mm.,경북강수량.mm.,서울경기강수량.mm.,전국강수량.mm.,전남강수량.mm.,전북강수량.mm.,제주강수량.mm.,충남강수량.mm.,충북강수량.mm.,강원영동평균기온...,강원영서평균기온...,경남평균기온...,경북평균기온...,서울경기평균기온...,전국평균기온...,전남평균기온...,전북평균기온...,제주평균기온...,충남평균기온...,충북평균기온...,유가,중국화폐종가,중국화폐시가,명절,태풍,대파거래량,대파가격,당근거래량,당근가격,토마토거래량,토마토가격,pos,neg,counts
0,4363,2016.01.04,월요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.4,3.0,6.7,6.0,2.3,5.3,7.2,5.5,12.0,4.0,4.0,37.22,182.14,180.85,0,0,994328.1,1716,558950.2,794,291057.0,1834,0,0,0
1,4363,2016.01.05,화요일,0.0,0.0,0.5,0.0,0.0,0.5,2.4,0.0,5.9,0.0,0.0,1.8,-2.3,2.4,1.5,-2.2,0.6,2.8,0.6,8.6,-0.9,-1.4,36.42,182.57,181.95,0,0,787716.0,1715,444353.7,763,194626.5,1833,0,0,0
2,4363,2016.01.06,수요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,-3.7,2.6,1.0,-1.1,0.7,3.6,1.2,8.3,-0.1,-1.7,34.23,183.10,182.40,0,0,793584.9,1620,396170.4,787,178811.0,1999,0,0,0
3,4363,2016.01.07,목요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,-3.7,2.4,0.7,-2.7,0.4,3.2,1.4,6.7,0.2,-1.9,33.75,181.60,182.91,0,0,733312.0,1545,241115.0,770,107346.3,1921,0,0,0
4,4363,2016.01.08,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.2,0.1,0.1,0.0,-0.3,-5.2,1.4,-0.3,-2.6,-0.5,2.2,0.5,5.7,-0.5,-2.9,33.55,183.12,181.44,0,0,797767.3,1488,359789.0,705,190624.0,2107,0,0,0


In [ ]:
# common
for i in range(len(full_data)) : 
  if full_data.iloc[i,1]+'.' in list(common_sum.index) : 
    date = full_data.iloc[i,1]
    full_data.iloc[i,36] += common_sum.loc[date+'.','pos']
    full_data.iloc[i,37] += common_sum.loc[date+'.','neg']
    full_data.iloc[i,38] += common_sum.loc[date+'.','word_counts']
  else : 
    continue

In [ ]:
tomato_final = full_data.copy()
daepa_final = full_data.copy()
carrot_final = full_data.copy()

In [ ]:
# tomato
for i in range(len(tomato_final)) : 
  if tomato_final.iloc[i,1]+'.' in list(tomato_sum.index) : 
    date = tomato_final.iloc[i,1]
    tomato_final.iloc[i,36] += tomato_sum.loc[date+'.','pos']
    tomato_final.iloc[i,37] += tomato_sum.loc[date+'.','neg']
    tomato_final.iloc[i,38] += tomato_sum.loc[date+'.','word_counts']
  else : 
    continue

In [ ]:
# daepa
for i in range(len(daepa_final)) : 
  if daepa_final.iloc[i,1]+'.' in list(daepa_sum.index) : 
    date = daepa_final.iloc[i,1]
    daepa_final.iloc[i,36] += daepa_sum.loc[date+'.','pos']
    daepa_final.iloc[i,37] += daepa_sum.loc[date+'.','neg']
    daepa_final.iloc[i,38] += daepa_sum.loc[date+'.','word_counts']
  else : 
    continue

In [ ]:
# carrot
for i in range(len(carrot_final)) : 
  if carrot_final.iloc[i,1]+'.' in list(carrot_sum.index) : 
    date = carrot_final.iloc[i,1]
    carrot_final.iloc[i,36] += carrot_sum.loc[date+'.','pos']
    carrot_final.iloc[i,37] += carrot_sum.loc[date+'.','neg']
    carrot_final.iloc[i,38] += carrot_sum.loc[date+'.','word_counts']
  else : 
    continue

In [ ]:
tomato_final

,외국인노동자수,date,요일,강원영동강수량.mm.,강원영서강수량.mm.,경남강수량.mm.,경북강수량.mm.,서울경기강수량.mm.,전국강수량.mm.,전남강수량.mm.,전북강수량.mm.,제주강수량.mm.,충남강수량.mm.,충북강수량.mm.,강원영동평균기온...,강원영서평균기온...,경남평균기온...,경북평균기온...,서울경기평균기온...,전국평균기온...,전남평균기온...,전북평균기온...,제주평균기온...,충남평균기온...,충북평균기온...,유가,중국화폐종가,중국화폐시가,명절,태풍,대파거래량,대파가격,당근거래량,당근가격,토마토거래량,토마토가격,pos,neg,counts
0,4363,2016.01.04,월요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.4,3.0,6.7,6.0,2.3,5.3,7.2,5.5,12.0,4.0,4.0,37.220000,182.140000,180.850000,0,0,994328.1,1716,558950.2,794,291057.0,1834,34,31,161
1,4363,2016.01.05,화요일,0.0,0.0,0.5,0.0,0.0,0.5,2.4,0.0,5.9,0.0,0.0,1.8,-2.3,2.4,1.5,-2.2,0.6,2.8,0.6,8.6,-0.9,-1.4,36.420000,182.570000,181.950000,0,0,787716.0,1715,444353.7,763,194626.5,1833,276,95,887
2,4363,2016.01.06,수요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,-3.7,2.6,1.0,-1.1,0.7,3.6,1.2,8.3,-0.1,-1.7,34.230000,183.100000,182.400000,0,0,793584.9,1620,396170.4,787,178811.0,1999,75,46,261
3,4363,2016.01.07,목요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,-3.7,2.4,0.7,-2.7,0.4,3.2,1.4,6.7,0.2,-1.9,33.750000,181.600000,182.910000,0,0,733312.0,1545,241115.0,770,107346.3,1921,45,21,151
4,4363,2016.01.08,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.2,0.1,0.1,0.0,-0.3,-5.2,1.4,-0.3,-2.6,-0.5,2.2,0.5,5.7,-0.5,-2.9,33.550000,183.120000,181.440000,0,0,797767.3,1488,359789.0,705,190624.0,2107,67,38,213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,375,2020.09.23,수요일,1.7,1.2,0.0,0.2,0.0,0.2,0.0,0.0,1.6,0.0,0.0,15.8,15.4,19.0,17.4,18.7,18.0,19.4,18.3,20.9,18.7,17.2,41.770000,171.850000,171.630000,1,1,734453.0,2216,408298.0,1792,88460.0,5529,24373,16758,92176
1457,375,2020.09.24,목요일,10.7,3.0,0.1,0.1,0.0,0.9,0.2,0.0,0.0,0.0,0.0,16.8,16.8,20.2,18.3,20.2,19.2,20.8,19.4,21.2,20.0,18.1,41.940000,171.170000,171.910000,1,1,770271.0,2380,423233.2,1565,142487.0,5587,24171,17082,91556
1458,375,2020.09.25,금요일,16.8,7.8,0.1,2.7,0.0,2.1,0.0,0.0,0.3,0.0,0.0,16.9,16.9,20.0,18.4,19.7,19.0,21.0,19.1,22.0,19.7,17.8,41.920000,172.050000,171.280000,1,0,781384.8,2239,400527.2,1773,152575.5,5596,19259,13515,73600
1459,375,2020.09.26,토요일,10.9,1.5,0.0,0.6,0.0,0.8,0.0,0.0,0.5,0.0,0.0,16.2,16.0,19.3,17.9,19.0,18.4,20.5,18.8,21.6,19.0,16.8,42.142386,171.733737,172.181827,1,0,810812.8,2126,187076.2,1405,176807.0,5262,4862,3406,18812


In [ ]:
# 필요했던 긍정, 부정, 단어수를 저장 완료 - 이를 토대로 나중에 감성지수 파생변수를 만들 수 있다
tomato_final.to_csv("/content/drive/MyDrive/선대팀_주제분석/tomato_final.csv", index = False)
daepa_final.to_csv("/content/drive/MyDrive/선대팀_주제분석/daepa_final.csv", index = False)
carrot_final.to_csv("/content/drive/MyDrive/선대팀_주제분석/carrot_final.csv", index = False)